# for the dear user

In [37]:
# please give me the string of path for the file "stage1_step1_export_bacdive_iso_table before cleaning.csv"
# for example  --->   r'C:\Users\kamy\Desktop\stage1_step1_export_bacdive_iso_table before cleaning.csv'
input_path = r'C:\Users\kamy\Desktop\stage1_step1_export_bacdive_iso_table before cleaning.csv'


# and again please give me the output path to save the final result
output_path = r'C:\Users\kamy\Desktop\OUTPUT.csv'

# importing all the packages we need

In [38]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

# STEP ONE

download a table from BacDive

i did it and the result is "saved as stage1_step1_export_bacdive_iso_table before cleaning.csv"

you can access it in git_hub repository

# STEP TWO

read stage1_step1_export_bacdive_iso_table before cleaning.csv

In [39]:
# read stage1_step1_export_bacdive_iso_table before cleaning.csv
bacDive = pd.read_csv(input_path)

In [40]:
# filling the table # replacing NaN with no in three colums (Category 1, Category 2, Category 3)
bacDive["Category 3"].fillna("#no", inplace = True)
bacDive["Category 2"].fillna("#no", inplace = True)
bacDive["Category 1"].fillna("#no", inplace = True) 

In [41]:
# the reason i used this code is to fill empty cells
# this code means --> check the IDs, if the ID of two consecutive rows are the same than fill the second row with the cells of first row

temporary_list =[]
x = len(bacDive['ID'])-1
for counter in range(0,x):
    if bacDive.iloc[counter,0] == bacDive.iloc[counter+1,0]:
        temporary_list.append(counter)
        bacDive.iloc[counter+1,1] = bacDive.iloc[counter,1]
        bacDive.iloc[counter+1,2] = bacDive.iloc[counter,2]
        bacDive.iloc[counter+1,3] = bacDive.iloc[counter,3]
        bacDive.iloc[counter+1,4] = bacDive.iloc[counter,4]
        bacDive.iloc[counter+1,5] = bacDive.iloc[counter,5]
        
        # we do not need the next code because we want to maintain the Tag data
        #bacDive.iloc[counter+1,7] = bacDive.iloc[counter,7] +  bacDive.iloc[counter+1,7]
        #bacDive.iloc[counter+1,8] = bacDive.iloc[counter,8] +  bacDive.iloc[counter+1,8]

In [42]:
######## we wont need this if we are going to maintain other Tags in future
######## this code is written to remove all the rows without a specific Tag(here : #Environmental)


#temporary_list =[]
#for counter in range (0,len(bacDive.index)):
    #if (bacDive.iloc[counter,6] != '#Environmental') :
        #temporary_list.append(counter)

######## we run this code at the end in order to keep all other category 1 tags 
#for i in temporary_list:
    #bacDive = bacDive.drop([i])

In [43]:
# now we have a dataframe whithout any tags other than #Environmental  but there are still some redundency, there are some rows with the same Species name
# here the goal is to merge rows with the same Species name


temporary_list =[]
x = len(bacDive['ID'])-1
for counter in range(0,x):
    if bacDive.iloc[counter,0] == bacDive.iloc[counter+1,0]:
        temporary_list.append(counter)
        bacDive.iloc[counter+1,7] = bacDive.iloc[counter,7] +  bacDive.iloc[counter+1,7]
        bacDive.iloc[counter+1,8] = bacDive.iloc[counter,8] +  bacDive.iloc[counter+1,8]
        


In [44]:
# and now we remove the duplicate row
for i in temporary_list:
    bacDive = bacDive.drop([i])

# STEP THREE

WEB SCRAPING from BacDive

In [45]:
# we want to creat URLs using the BacDive IDs
all_IDs = bacDive['ID']

producing links for web scrapping

In [46]:
def get_ID_give_URL(ID):
    url = 'https://bacdive.dsmz.de/strain/' + str(ID)
    return url

reading html

In [47]:
def read_html(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        return response.text
    return None
#CODE = 200 means the url is availible

In [59]:
# my regex to extract temperature data from BacDive
# in future i should improve this regex 
my_regex_temperature = re.compile("(Ref\.\:.\#\d+)\]\<\/a\>\<\/td\>\s\<td\>\<\/td\>\s\<td.class\=\"border\_rightfree\ textalign\_right\"\>\<\/td\>\s\<td.class\=\"border\_leftfree\"\>(\w+)\<\/td\>\s\<td.class\=\"border_leftfree textalign_center\"\>(\d{2}\-\d{2}|\d{2}\.\d{1}|\d{2})")

# my regex to extract pH data from BacDive
# in future i should improve this regex 
x=str('(Ref\.\:.\#\d+)\]\<\/a\>\<\/td\>\\n\<td\>\<\/td\>\\n\<td\sclass=\"border\_rightfree\stextalign\_right\"\>\<\/td\>\\n\<td\sclass=\"border\_leftfree\"\>(\w+)\<\/td\>\\n\<td\sclass=\"valigntop\sborder\stextalign\_center\"\>(\d+\.\d+\-\d\.\d+)')
my_regex_pH = re.compile(x)

In [49]:
my_data_frame = pd.DataFrame()


for ID in all_IDs:
    url = get_ID_give_URL(ID)
    html_doc = read_html(url)
    
    
    if html_doc is None:
        print("Something went wrong!!!  the following url seems to be wrong   ; " , url)
    
    soup = BeautifulSoup(html_doc, "lxml")
    list_of_extracted_data = [ID]
    
    
    #first step ==> extracting phylogeny data
    tag = "valigntop paddingright"
    data = soup.find_all("td", class_= tag)
    for td in data:
        list_of_extracted_data.append(td.text)
        if len(list_of_extracted_data) == 9:
            break
        
    #second step ==> extracting temp data
    soup = str(soup)
    temperature_data = my_regex_temperature.findall(soup)
    list_of_extracted_data = list_of_extracted_data +temperature_data
    while len(list_of_extracted_data) != 16:
        list_of_extracted_data.append("")
    
    
    
    #third step ==> extracting phylogeny data
    
    soup = str(soup)
    pH_data = my_regex_pH.findall(soup)
    list_of_extracted_data = list_of_extracted_data + pH_data
    while len(list_of_extracted_data) != 22:
        list_of_extracted_data.append("")
    my_data_frame[ID] = pd.Series(list_of_extracted_data)
    
   


# transpose the dataframe
my_data_frame = my_data_frame.T
# naming columns
my_data_frame = my_data_frame.rename(columns={0: 'ID',  1: 'Last LPSN update', 2: 'Domain', 3: 'Phylum', 4: 'Class', 5: 'Order', 6: 'Family', 7: 'Genus', 8: 'Species', 9: 'temperature Ref 1', 10: 'temperature Ref 2', 11: 'temperature Ref 3', 12: 'temperature Ref 4', 13: 'temperature Ref 5', 14: 'temperature Ref 6', 15: 'temperature Ref 7', 16: 'pH 1', 17: 'pH 2', 18: 'pH 3', 19: 'pH 4', 20: 'pH 5', 21: 'pH 6'})

# STEP FOUR

another cleaning and filling step

In [57]:
# fill the cells and replacing "NaN" with "#no"
# this will clean the dataframe for future use
my_data_frame["temperature Ref 1"].fillna("#no", inplace = True)
my_data_frame["temperature Ref 2"].fillna("#no", inplace = True)
my_data_frame["temperature Ref 3"].fillna("#no", inplace = True)
my_data_frame["temperature Ref 4"].fillna("#no", inplace = True)
my_data_frame["temperature Ref 5"].fillna("#no", inplace = True)
my_data_frame["temperature Ref 6"].fillna("#no", inplace = True)
my_data_frame["temperature Ref 7"].fillna("#no", inplace = True)
my_data_frame["pH 1"].fillna("#no", inplace = True)
my_data_frame["pH 2"].fillna("#no", inplace = True)
my_data_frame["pH 3"].fillna("#no", inplace = True)
my_data_frame["pH 4"].fillna("#no", inplace = True)
my_data_frame["pH 5"].fillna("#no", inplace = True)
my_data_frame["pH 6"].fillna("#no", inplace = True)

In [51]:
#### in order to know the species with no temperature data  ####
list_index_no_temp = []
list_no_temp_species_ID = []

for counter in range (0,len(my_data_frame)):
    if (my_data_frame.iloc[counter,10] == "#no"):
        list_index_no_temp.append(counter)
        list_no_temp_species_ID.append(my_data_frame.iloc[counter,0])
        
# give me an overview please
print('until now, there are', str(len(list_no_temp_species_ID)) , 'species with no temperature data and you can see the list of IDs with no temp data in this: list_no_temp_species_ID')


until now, there are 0 species with no temperature data and you can see the list of IDs with no temp data in this: list_no_temp_species_ID


# STEP FIVE

concat all the previous dataframes and producing an output

In [52]:
# making to dataframes look the same , so we can use concat()
s = pd.Series(range(len(bacDive)))
bacDive = bacDive.set_index([s])
###########################################
s = pd.Series(range(len(my_data_frame)))
my_data_frame = my_data_frame.set_index([s])
###########################################
result = pd.concat([bacDive, my_data_frame], axis=1)
###########################################
result.to_csv(output_path)

# STAGE 2

# STAGE 2

# STAGE 2

# STAGE 2

# STAGE 2